In [31]:
import pandas as pd
from fuzzywuzzy import fuzz, process
import re
import numpy as np
import os

import time

In [32]:
country = "CL"
geo2_nq = "CL_comuna"
geo1_nq = "CL_provincia"

In [33]:
census = pd.read_csv("../Data/"+country+"/sample/"+"ipumsi_00020.csv")

In [34]:
netquest = pd.read_csv("../Data/"+country+"/panel/"+country+"_netquest-panel.csv")

In [35]:
ipums_geo2 = pd.read_csv('../Data/CL/sample/ipums_codebook_GEO2_CL2002.csv',encoding='Latin1', names=['code','name'], skiprows=1)
ipums_geo1 = pd.read_csv('../Data/'+country+'/sample/ipums_codebook_GEO1_CL2002.csv',encoding='Latin1', names=['code','name'], skiprows=1)

In [36]:
ipums_geodf = census[['GEO1_CL2002','GEO2_CL2002']]
ipums_geodf.columns = ['geo1_code','geo2_code']
ipums_geodf.drop_duplicates(inplace=True)

ipums_geodf['geo1_name'] = ipums_geodf.merge(ipums_geo1, 
                                             how='left', 
                                             left_on = "geo1_code", 
                                             right_on = 'code', 
                                             copy=False)['name'].values

ipums_geodf['geo2_name'] = ipums_geodf.merge(ipums_geo2, 
                                             how='left', 
                                             left_on = "geo2_code", 
                                             right_on = 'code', 
                                             copy=False)['name'].values

C:\Users\schadem\AppData\Local\Continuum\anaconda3\envs\idb\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\schadem\AppData\Local\Continuum\anaconda3\envs\idb\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\schadem\AppData\Local\Continuum\anaconda3\envs\idb\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

We have to get the melting action going here.

In [37]:
ipums_geodf = ipums_geodf.geo2_name.str.split(',').apply(pd.Series) \
    .merge(ipums_geodf, right_index = True, left_index = True) \
    .drop(["geo2_name"], axis = 1) \
    .melt(id_vars = [k for k in ipums_geodf.columns if not (type(k)==int)|(k=='geo2_name')], value_name = "geo2_name") \
    .drop("variable", axis = 1) \
    .dropna(subset=['geo2_name'])
ipums_geodf.geo2_name = ipums_geodf.geo2_name.str.strip()

In [38]:
ipums_geodf = ipums_geodf.geo1_name.str.split(',').apply(pd.Series) \
    .merge(ipums_geodf, right_index = True, left_index = True) \
    .drop(["geo1_name"], axis = 1) \
    .melt(id_vars = [k for k in ipums_geodf.columns if not (type(k)==int)|(k=='geo1_name')], value_name = "geo1_name") \
    .drop("variable", axis = 1) \
    .dropna(subset=['geo1_name'])
ipums_geodf.geo1_name = ipums_geodf.geo1_name.str.strip()

Now to Netquest.

In [39]:
province_labels = pd.read_excel("../Data/CL/panel/provincia_dict.xlsx", 
                                encoding='Latin1', header=None)
nq_geodf = netquest[[geo1_nq,geo2_nq,geo2_nq+'_name']]
nq_geodf.columns = ['geo1_code','geo2_code','geo2_name']

#the following is Chile-specific
nq_geodf['geo1_name'] = netquest.CL_provincia.fillna(-99
                                                 ).astype(int
                                                ).replace(province_labels[1].values,
                                                   province_labels[0].values
                                                  )
nq_geodf.drop_duplicates(inplace=True)

C:\Users\schadem\AppData\Local\Continuum\anaconda3\envs\idb\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\schadem\AppData\Local\Continuum\anaconda3\envs\idb\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


Finding the duplicates in nq_geodf.geo2_code, and keeping the combinations that occur most often (which are quite clearly the good ones).

In [40]:
nq_geodf['count'] = nq_geodf.apply(lambda r: sum((netquest[geo1_nq]==r['geo1_code'])&(netquest[geo2_nq]==r['geo2_code'])),
                                   axis=1)
nq_geodf = nq_geodf.sort_values(['geo2_code','count']) \
    [~(nq_geodf.sort_values(['geo2_code','count']).duplicated('geo2_code',keep='last'))]
# tmp = nq_geodf[~(nq_geodf.sort_values(['geo2_code','count']).duplicated('geo2_code',keep='last'))]
#nq_geodf = nq_geodf[~((nq_geodf.geo2_code.duplicated(keep=False))&(nq_geodf['count']<2))]

C:\Users\schadem\AppData\Local\Continuum\anaconda3\envs\idb\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Manual fixes, as found necessary in the next step...

In [41]:
nq_geodf.loc[nq_geodf.geo1_name=="Marga Marga",'geo1_name'] = 'Valparaíso'
nq_geodf.loc[nq_geodf.geo1_name=="Aysen",'geo1_name'] = 'Aisén'

nq_geodf.loc[nq_geodf.geo1_name=="Diguillin",'geo1_name'] = 'Ñuble'
nq_geodf.loc[nq_geodf.geo1_name=="Itata",'geo1_name'] = 'Ñuble'
nq_geodf.loc[nq_geodf.geo1_name=="Punilla",'geo1_name'] = 'Ñuble'

ipums_geodf.loc[ipums_geodf.geo2_name.str.contains("San Fernando"),
                'geo1_name'] = "Colchagua"
ipums_geodf.loc[ipums_geodf.geo2_name.str.contains("Chimbarongo"),
                'geo1_name'] = "Colchagua"
ipums_geodf.loc[ipums_geodf.geo2_name.str.contains("Palmilla"),
                'geo1_name'] = "Colchagua"
ipums_geodf.loc[ipums_geodf.geo2_name.str.contains("Nancagua"),
                'geo1_name'] = "Colchagua"
ipums_geodf.loc[ipums_geodf.geo2_name.str.contains("Chépica"),
                'geo1_name'] = "Colchagua"
ipums_geodf.loc[ipums_geodf.geo2_name.str.contains("Pumanque"),
                'geo1_name'] = "Colchagua"
ipums_geodf.loc[ipums_geodf.geo2_name.str.contains("Placilla"),
                'geo1_name'] = "Colchagua"
ipums_geodf.loc[ipums_geodf.geo2_name.str.contains("Lolol"),
                'geo1_name'] = "Colchagua"
ipums_geodf.loc[ipums_geodf.geo2_name.str.contains("Peralillo"),
                'geo1_name'] = "Colchagua"
ipums_geodf.loc[ipums_geodf.geo2_name.str.contains("Santa Cruz"),
                'geo1_name'] = "Colchagua"

ipums_geodf.loc[ipums_geodf.geo2_name=="Pozo Almonte",'geo1_name'] = 'Tamarugal'
ipums_geodf.loc[ipums_geodf.geo2_name.str.contains("Lago Ranco"),'geo1_name'] = 'Ranco'
ipums_geodf.loc[ipums_geodf.geo2_name.str.contains("Futrono"),
                'geo1_name'] = "Ranco"
ipums_geodf.loc[ipums_geodf.geo2_name.str.contains("Río Bueno"),
                'geo1_name'] = "Ranco"
ipums_geodf.loc[ipums_geodf.geo2_name.str.contains("La Unión"),
                'geo1_name'] = "Ranco"

ipums_geodf.loc[ipums_geodf.geo2_name.str.contains("Olmué"),'geo1_name'] = 'Valparaíso'
ipums_geodf.loc[ipums_geodf.geo2_name=="Limache",'geo1_name'] = 'Valparaíso'
nq_geodf.loc[nq_geodf.geo2_name=="Limache",'geo1_name'] = 'Valparaíso'
nq_geodf.loc[nq_geodf.geo2_name=="San Antonio",'geo1_name'] = 'San Antonio'

nq_geodf.loc[nq_geodf.geo2_name=="Alto Hospicio",'geo2_name'] = 'Iquique'
nq_geodf.loc[nq_geodf.geo2_name=="Hualpén",'geo2_name'] = 'Talcahuano'
nq_geodf.loc[nq_geodf.geo2_name=="Cholchol",'geo2_name'] = 'Nueva Imperial'
nq_geodf.loc[nq_geodf.geo2_name=="Alto Bío-Bí",'geo2_name'] = 'Santa Bárbara'
nq_geodf.loc[nq_geodf.geo2_name=="Antofagasta",'geo1_name'] = 'Antofagasta'
nq_geodf.loc[nq_geodf.geo2_name=="Chillán",'geo1_name'] = 'Ñuble'
nq_geodf.loc[nq_geodf.geo2_name=="Colbún",'geo1_name'] = 'Linares'
nq_geodf.loc[nq_geodf.geo2_name=="Coquimbo",'geo1_name'] = 'Elqui'
nq_geodf.loc[nq_geodf.geo2_name=="Maipú",'geo1_name'] = 'Santiago'
nq_geodf.loc[nq_geodf.geo2_name=="Quilicura",'geo1_name'] = 'Santiago'
nq_geodf.loc[nq_geodf.geo2_name=="Maipú",'geo1_name'] = 'Santiago'
nq_geodf.loc[nq_geodf.geo2_name=="Puente Alto",'geo1_name'] = 'Cordillera'
nq_geodf.loc[nq_geodf.geo2_name=="Pirque",'geo1_name'] = 'Cordillera'
nq_geodf.loc[nq_geodf.geo2_name=="Peñaflor",'geo1_name'] = 'Talagante'
nq_geodf.loc[nq_geodf.geo2_name=="Buin",'geo1_name'] = 'Maipo'
nq_geodf.loc[nq_geodf.geo2_name=="San Bernardo",'geo1_name'] = 'Maipo'

ipums_geodf.loc[ipums_geodf.geo2_name.isin(["Talcahuano",
                                            "Concepción",
                                            "Coronel",
                                            "Lota"
                                           ]),
                'geo1_name'] = "Concepción"

nq_geodf.loc[nq_geodf.geo2_name.isin(["Talcahuano",
                                            "Concepción",
                                            "Coronel",
                                            "Lota"
                                           ]),
                'geo1_name'] = "Concepción"

nq_geodf.loc[nq_geodf.geo2_name.isin(["Colchane",'Pica','Huara',"Camiña",
                                           ]),
                'geo1_name'] = "Iquique"

Next, we'll have to merge together the two names for matching. ACtually, I don't think that will work given the obnoxious abbreviations... I'll have to see about this.

In [42]:
def fuzzy_join(df1, df2, varname, subset=False, cutoff=90):
    '''Returns a dataframe the length of df1, with the matches on the given variable
    
        Assumes the var is named the same in both dataframes.
        
        subset is a tuple or list of length 2 whose first element is a string indicating which
        variable in df1 has to be equal to which variable in df2 (specified by 2nd element)
        '''
    #figure out if there is a constraint
    if not subset:
        #match #1
        ratio_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname].astype(str).drop_duplicates(),
                                             scorer=fuzz.ratio, limit=2
                                            ), axis=1)
        #match #2
        parti_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname].astype(str).drop_duplicates(),
                                             scorer=fuzz.partial_ratio, limit=2
                                            ), axis=1)
        
    else:
        #match #1
        ratio_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname][df2[subset[1]]==d[subset[0]]].astype(str).drop_duplicates(),
                                             scorer=fuzz.ratio, limit=2
                                            ), axis=1)
        #match #2
        parti_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname][df2[subset[1]]==d[subset[0]]].astype(str).drop_duplicates(),
                                             scorer=fuzz.partial_ratio, limit=2
                                            ), axis=1)    

    
    # different match cases
    #morRatioMatch = ratio_matches.apply(lambda l: (l[0][1]==100)&(l[1][1]==100))
    oneRatioMatch = ratio_matches.apply(lambda l: (l[0][1]>=cutoff))#&(l[1][1]<100))
    noRatioMatch = ratio_matches.apply(lambda l: (l[0][1]<cutoff))

    #morPartiMatch = parti_matches.apply(lambda l: (l[0][1]==100)&(l[1][1]==100))
    onePartiMatch = parti_matches.apply(lambda l: (l[0][1]>=cutoff))#&(l[1][1]<100))
    noPartiMatch = parti_matches.apply(lambda l: (l[0][1]<cutoff))
    
    # pick out what's better
    matches = pd.Series([(np.nan,np.nan,np.nan)]*len(df1),index=df1.index)
    matches.loc[oneRatioMatch] = ratio_matches.loc[oneRatioMatch].apply(lambda l: l[0])
    matches.loc[(noRatioMatch&onePartiMatch)] = parti_matches.loc[(noRatioMatch&onePartiMatch)]\
        .apply(lambda l: l[0])
    matches.loc[(noRatioMatch&noPartiMatch)] = parti_matches.loc[(noRatioMatch&noPartiMatch)]\
        .apply(lambda l: l[0])
    
    df = pd.DataFrame()
    df['name'] = matches.apply(lambda l: l[0])
    df['score'] = matches.apply(lambda l: l[1])
    df['index'] = matches.apply(lambda l: l[2])
    df['parti_matches'] = parti_matches
    df['ratio_matches'] = ratio_matches
    
    return df

Use the new tool!

In [43]:
nq_geodf[['geo1_match_name','geo1_match_score','geo1_match_index']] \
= fuzzy_join(nq_geodf, ipums_geodf, 'geo1_name')[['name','score','index']]

In [44]:
nq_geodf[['geo2_match_name','geo2_match_score','geo2_match_index']] \
= fuzzy_join(nq_geodf, ipums_geodf, 'geo2_name', 
             subset=['geo1_match_name','geo1_name']
            )[['name','score','index']]

## De-duping

Some testing.

What can we eliminate? Where geo_2code is duplicated and...
* where geo2_code and geo2_match_code are duplicated across dupes, just de-dupe--doesn't matter which we use
* drop any that have NaN in their geo-s
* one of the geo2_match_names is an exact match, throw out any others
* if there are two contenders, check which code+geo1_name+geo2_name combo has the majority--thankfully doesn't seem to be necessary here

In [45]:
tmp = nq_geodf.copy()

In [46]:
# # if we have the same mapping between netquest geo2 and ipums geo2, doesn't matter which we pick
# tmp = tmp.drop_duplicates(subset=['geo2_code','geo2_match_index'])
# # ones that have NA geographies are useless in this context, so drop them first
# tmp = tmp[(~tmp.geo1_code.isna())& (~tmp.geo2_code.isna())]

# dupes = tmp[tmp.duplicated('geo2_code')].geo2_code
# for code in dupes: 
#     # if one of the geo2_match_names is an exact match, throw out any others
#     if sum(tmp.loc[tmp.geo2_code==code,"geo2_name"]==tmp.loc[tmp.geo2_code==code,"geo2_match_name"])>0:
#         tmp = tmp[~((tmp.geo2_code==code) & (tmp.geo2_name!=tmp.geo2_match_name))]
#     # if there are still duplicates, use the one that has the most prevalent pairing of geo1 & geo2
# #     id_of_most_common = tmp.loc[tmp.geo2_code==code,"geo1_code"].apply(
# #         lambda c: sum((netquest[geo1_nq]==c) & (netquest[geo2_nq]==code))
# #     ).idxmax()
# #     correct_geo1 = tmp.loc[id_of_most_common,"geo1_code"]
# #     tmp = tmp[~((tmp.geo2_code==code) & ~(tmp.geo1_code==correct_geo1))]
        

**What is still an issue??**

In [47]:
tmp[tmp.duplicated('geo2_code', keep=False)].sort_values('geo2_code')\
[["geo2_code","geo1_name","geo1_match_name","geo2_name","geo2_match_name","geo2_match_score","geo2_match_index"]]

,geo2_code,geo1_name,geo1_match_name,geo2_name,geo2_match_name,geo2_match_score,geo2_match_index


Some testing...

In [48]:
tmp[tmp.geo2_match_score<90][['geo1_name',
                                        'geo1_match_name',
                                        'geo2_name',
                                        'geo2_match_name',
                                        'geo2_match_score']
                                      ].sort_values('geo1_match_name'
                                      )

,geo1_name,geo1_match_name,geo2_name,geo2_match_name,geo2_match_score
3189,Aisén,Aisén,Aysen,Aisén,60
1141,Cachapoal,Cachapoal,Doñihue,Doñigue,86
159,Cachapoal,Cachapoal,Machalí,Machali,86
1385,Cachapoal,Cachapoal,Requínoa,Requinoa,88
7370,Curicó,Curicó,Hualañé,Hualañe,86
47785,El Loa,El Loa,Ollagüe,Ollague,86
1697,Linares,Linares,Longaví,Longavi,86
2855,Llanquihue,Llanquihue,Maullín,Maullin,86
350,Malleco,Malleco,Traiguén,Traiguen,88
207,Melipilla,Melipilla,Curacaví,Curacavi,88


Success! This can be the geo-df.

In [49]:
nq_geodf = tmp

Get a crosswalk for these!

In [50]:
has_ipums_geo = nq_geodf.geo2_match_index.notna()

In [51]:
nq_geodf['IPUMS_geo2_code'] = np.nan

nq_geodf.loc[has_ipums_geo,'IPUMS_geo2_code'] = nq_geodf[has_ipums_geo]\
                            .geo2_match_index\
                            .astype(int)\
                            .apply(
                                lambda i: ipums_geodf.loc[i,'geo2_code']
                            .astype(int)
)

We want to replace each nq_geo2 with the _code_ for the corresponding census geography.



Read in the municipio-centroids, and hopefully attach them here.

In [52]:
geo2_centroids = pd.read_csv('../Data/'+country+'/geography/'+country+'_geo_centroids.csv', encoding='latin1')

In [53]:
nq_geodf = nq_geodf.merge(geo2_centroids[['ADMIN_NAME','Y','X','IPUM2002']], 
               left_on='IPUMS_geo2_code',
               right_on="IPUM2002",
               how='left'
              ).drop('IPUM2002', axis=1)

In [54]:
panel_geo = netquest.merge(nq_geodf[['X','Y','geo2_code']],
               left_on=geo2_nq,
               right_on='geo2_code',
               how='left'
              ).drop('geo2_code',axis=1)

In [55]:
panel_geo.shape[0]==netquest.shape[0]

True

In [56]:
panel_geo.to_csv('../Data/'+country+'/panel/'+country+'_netquest-panel_geo.csv')

In [57]:
census_geo = census.merge(geo2_centroids[['ADMIN_NAME','X',"Y",'IPUM2002']],
                          left_on = 'GEO2_CL2002',
                          right_on='IPUM2002',
                          how='left'
                         ).drop('IPUM2002',axis=1)

In [58]:
census_geo.shape[0]==census.shape[0]

True

In [59]:
census_geo.to_csv('../Data/'+country+'/sample/'+country+'_ipums-census_geo.csv')